In [3]:
import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten
from keras.optimizers import Adam


class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95   # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01  # exploration will not decay futher
        self.epsilon_decay = 0.990
        self.learning_rate = 0.0005
        self.model = self._build_model()
        self.weight_backup = 'model_weights.h5'
    '''
    def _build_model(self):
        model = Sequential(),
        model.add(Conv2D(32, kernel_size=8, subsample=(4, 4), activation='relu', padding='same', input_shape=4))  #80*80*4
        model.add(Conv2D(64, kernel_size=4, subsample=(2, 2), activation='relu', padding='same'))
        model.add(Conv2D(64, kernel_size=3, subsample=(1, 1), activation='relu', padding='same'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(2, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model
    '''

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def save_model(self):
            self.model.save(self.weight_backup)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def remember(self, state, action, reward, new_state, done):
        self.memory.append((state, action, reward, new_state, done))

    def memory_replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        Sample = random.sample(self.memory, batch_size)
        for state, action, reward, new_state, done in Sample:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(new_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [4]:
import sys
import gym
from gym import wrappers

batch_size = 32
#episodes = sys.argv[1] if len(sys.argv) > 1 else 5000
#env_name = sys.argv[2] if len(sys.argv) > 2 else "Pong-v0"

episodes = 1500
env_name = "CartPole-v0"

env = gym.make(env_name)

env = wrappers.Monitor(env, env_name, force=True)

agent = Agent(env.observation_space.shape[0], env.action_space.n)

for i_episodes in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, env.observation_space.shape[0]])
    index = 0
    done = False
    while not done:
        env.render()
        action = agent.act(state)
        new_state, reward, done, info = env.step(action)
        new_state = np.reshape(new_state, [1, env.observation_space.shape[0]])
        agent.remember(state, action, reward, new_state, done)
        state = new_state
        index += 1
    agent.memory_replay(batch_size)
    if done:
        print("{} episode, score = {} ".format(i_episodes + 1, index + 1))
        agent.save_model()

env.close()
gym.upload(env_name, api_key='sk_WRCITkqmTJKYB9hvBk5tPA')

[2017-08-12 20:18:34,617] Making new env: CartPole-v0
[2017-08-12 20:18:34,630] Clearing 24 monitor files from previous run (because force=True was provided)
[2017-08-12 20:18:34,727] Starting new video recorder writing to /home/z0m6ie/Documents/GitHub/Machine_Learning_Projects/deep-Q-learning/Untitled Folder/CartPole-v0/openaigym.video.1.9886.video000000.mp4
[2017-08-12 20:18:35,480] Starting new video recorder writing to /home/z0m6ie/Documents/GitHub/Machine_Learning_Projects/deep-Q-learning/Untitled Folder/CartPole-v0/openaigym.video.1.9886.video000001.mp4


1 episode, score = 25 
2 episode, score = 37 
3 episode, score = 22 
4 episode, score = 29 
5 episode, score = 15 
6 episode, score = 13 
7 episode, score = 13 
8 episode, score = 26 


[2017-08-12 20:18:37,906] Starting new video recorder writing to /home/z0m6ie/Documents/GitHub/Machine_Learning_Projects/deep-Q-learning/Untitled Folder/CartPole-v0/openaigym.video.1.9886.video000008.mp4


9 episode, score = 14 
10 episode, score = 30 
11 episode, score = 16 
12 episode, score = 13 
13 episode, score = 14 
14 episode, score = 26 
15 episode, score = 14 
16 episode, score = 14 
17 episode, score = 11 
18 episode, score = 16 
19 episode, score = 19 
20 episode, score = 19 
21 episode, score = 16 
22 episode, score = 27 
23 episode, score = 21 
24 episode, score = 13 
25 episode, score = 26 
26 episode, score = 16 
27 episode, score = 26 


[2017-08-12 20:18:41,729] Starting new video recorder writing to /home/z0m6ie/Documents/GitHub/Machine_Learning_Projects/deep-Q-learning/Untitled Folder/CartPole-v0/openaigym.video.1.9886.video000027.mp4


...

1000 episode, score = 86 
1001 episode, score = 41 
1002 episode, score = 45 
1003 episode, score = 40 
1004 episode, score = 23 
1005 episode, score = 33 
1006 episode, score = 73 
1007 episode, score = 66 
1008 episode, score = 26 
1009 episode, score = 43 
1010 episode, score = 70 
1011 episode, score = 33 
1012 episode, score = 33 
1013 episode, score = 34 
1014 episode, score = 31 
1015 episode, score = 25 
1016 episode, score = 70 
1017 episode, score = 39 
1018 episode, score = 38 
1019 episode, score = 50 
1020 episode, score = 46 
1021 episode, score = 42 
1022 episode, score = 57 
1023 episode, score = 37 
1024 episode, score = 29 
1025 episode, score = 40 
1026 episode, score = 32 
1027 episode, score = 38 
1028 episode, score = 35 
1029 episode, score = 45 
1030 episode, score = 36 
1031 episode, score = 25 
1032 episode, score = 20 
1033 episode, score = 16 
1034 episode, score = 15 
1035 episode, score = 17 
1036 episode, score = 14 
1037 episode, score = 13 
1038 episode

1309 episode, score = 190 
1310 episode, score = 197 
1311 episode, score = 195 
1312 episode, score = 201 
1313 episode, score = 200 
1314 episode, score = 201 
1315 episode, score = 201 
1316 episode, score = 201 
1317 episode, score = 201 
1318 episode, score = 201 
1319 episode, score = 182 
1320 episode, score = 189 
1321 episode, score = 201 
1322 episode, score = 201 
1323 episode, score = 198 
1324 episode, score = 185 
1325 episode, score = 200 
1326 episode, score = 201 
1327 episode, score = 201 
1328 episode, score = 201 
1329 episode, score = 183 
1330 episode, score = 201 
1331 episode, score = 201 
1332 episode, score = 201 
1333 episode, score = 201 
1334 episode, score = 201 
1335 episode, score = 201 
1336 episode, score = 201 
1337 episode, score = 201 
1338 episode, score = 201 
1339 episode, score = 201 
1340 episode, score = 201 
1341 episode, score = 194 
1342 episode, score = 201 
1343 episode, score = 201 
1344 episode, score = 178 
1345 episode, score = 201 
1

[2017-08-12 20:35:33,384] Finished writing results. You can upload them to the scoreboard via gym.upload('/home/z0m6ie/Documents/GitHub/Machine_Learning_Projects/deep-Q-learning/Untitled Folder/CartPole-v0')
[2017-08-12 20:35:33,394] [CartPole-v0] Uploading 1500 episodes of training data


1500 episode, score = 201 


[2017-08-12 20:35:34,742] [CartPole-v0] Uploading videos of 11 training episodes (30115 bytes)
[2017-08-12 20:35:35,126] [CartPole-v0] Creating evaluation object from CartPole-v0 with learning curve and training video
[2017-08-12 20:35:35,542] 
****************************************************
You successfully uploaded your evaluation on CartPole-v0 to
OpenAI Gym! You can find it at:

    https://gym.openai.com/evaluations/eval_lbEN48CuTTSozMLoSgIlLQ

****************************************************


In [ ]:
new_state.shape

In [7]:
env.reset()
env.close()

In [ ]:
agent.memory